<h1>Monsoon Powertool python

In [89]:
import sys
sys.executable

'/home/joonb/anaconda3/bin/python'

In [90]:
import monsoon #importing monsoon.py
import numpy as np 
import time

In [93]:
MAIN_VOLTAGE=4.0
VOLTAGE_PADDING=1.0

NUM_SAMPLES = 45000
PADDING_FRONT = 20000
PADDING_BACK = 10000

USB_PASSTHROUGH_AUTO=2

In [94]:
#if monsoon.__init__ says permission denied, type this in shell and reboot 
#$ usermod -a -G dialout $USER

mon = monsoon.Monsoon()
#mon.GetStatus()
mon.SetVoltage(MAIN_VOLTAGE)

mon.SetUsbPassthrough(USB_PASSTHROUGH_AUTO)

In [144]:
def measure_power_short():
    data = []
    mon.StartDataCollection()
    
    while len(data) < 3000:
        d = mon.CollectData()
        if (d is None or
            len(d) == 0):
            continue
        data.extend(d[0])
        
    mon.StopDataCollection()
    
    main_power_mW = list(x * 1000.0 * MAIN_VOLTAGE for x in data)
    
    return main_power_mW[PADDING_FRONT:-PADDING_BACK]

In [193]:
def measure_power():
    mon.StopDataCollection()
    mon.StopDataCollection()
    mon.StopDataCollection()
    
    data = []
    mon.StartDataCollection()
    
    while len(data) < NUM_SAMPLES:
        d = mon.CollectData()
        if (d is None or
            len(d) == 0):
            continue
        data.extend(d[0])
        
    mon.StopDataCollection()
    mon.StopDataCollection()
    mon.StopDataCollection()
    
    main_power_mW = list(x * 1000.0 * MAIN_VOLTAGE for x in data)
    
    return main_power_mW[PADDING_FRONT:-PADDING_BACK]

In [194]:
mon.StopDataCollection()

In [195]:
measure_power()

wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58


[421.30809711228056,
 417.2325653385221,
 418.2514482819617,
 411.1192676778844,
 422.3269800557202,
 415.1947994516429,
 419.2703312254014,
 420.28921416884094,
 415.1947994516429,
 427.42139477291823,
 420.28921416884094,
 483.45995666209706,
 491.61102020961397,
 476.3277760580197,
 476.3277760580197,
 457.9878830761067,
 421.30809711228056,
 416.2136823950825,
 419.2703312254014,
 416.2136823950825,
 421.30809711228056,
 415.1947994516429,
 419.2703312254014,
 416.2136823950825,
 415.1947994516429,
 421.30809711228056,
 417.2325653385221,
 421.30809711228056,
 416.2136823950825,
 416.2136823950825,
 422.3269800557202,
 414.1759165082033,
 421.30809711228056,
 414.1759165082033,
 417.2325653385221,
 422.3269800557202,
 426.4025118294786,
 421.30809711228056,
 415.1947994516429,
 419.2703312254014,
 416.2136823950825,
 421.30809711228056,
 417.2325653385221,
 416.2136823950825,
 419.2703312254014,
 419.2703312254014,
 423.3458629991598,
 414.1759165082033,
 415.1947994516429,
 419.27

<h1>ADB

In [196]:
#!pip install --upgrade pip
#!pip install adb
import subprocess

In [197]:
#in case of adb connection error use this

def wait_adb_connection():
    import time
    retry = 0
    while len(subprocess.check_output ("adb devices" , shell=True).decode('utf-8').strip().split('\n')) == 1:
        time.sleep(0.5)
        retry += 1
        if retry >= 15:
            mon.StopDataCollection()
            mon.StopDataCollection()
            
            measure_power_short()
            
            mon.StopDataCollection()
            mon.StopDataCollection()
            
            retry = 0

<h3>Initialize device prior to experiments

In [153]:
#mtp doesn't turn off. on samsung device

#subprocess.check_output ("adb shell su -c 'pm disable com.samsung.android.MtpApplication'" , shell=True).decode('utf-8')

In [154]:
wait_adb_connection()

In [155]:
#unplug the usb (pretending)
subprocess.check_output ("adb shell dumpsys battery unplug" , shell=True).decode('utf-8')

''

In [156]:
subprocess.check_output ("adb shell dumpsys battery set level 90" , shell=True).decode('utf-8')

''

subprocess.check_output ("adb shell \"su -c 'echo 0 > /sys/power/enable_dm_hotplug'\"" , shell=True).decode('utf-8')

In [ ]:
#when device turn off the screen, the CPU goes to sleep as well.
#to prevent this use the below command
subprocess.check_output ("adb shell \"su -c 'echo amoled > /sys/power/wake_lock'\"" , shell=True).decode('utf-8')

In [158]:
#turning off all cpu cores except for cpu 0.

for cpu_id in [3,2,1]:
    cmd = "adb shell cat /sys/devices/system/cpu/cpu%d/online" % cpu_id
    state = int(subprocess.check_output (cmd , shell=True).decode('utf-8').strip())
    
    if state == 1:
        cmd = "adb shell \"su -c 'echo 0 > /sys/devices/system/cpu/cpu%d/online'\"" % cpu_id
        subprocess.check_output (cmd , shell=True).decode('utf-8')

In [159]:
#using userspace governor

subprocess.check_output("adb shell \"su -c 'echo userspace > /sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'\"" , shell=True).decode('utf-8')


# min_frequency_str = subprocess.check_output ("adb shell cat /sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq", shell=True).decode('utf-8')
# min_frequency = int(min_frequency_str.strip())

# min_frequency

# cmd = "adb shell \"su -c 'echo %d > /sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'\"" % min_frequency
# subprocess.check_output (cmd , shell=True).decode('utf-8')

''

In [160]:
#checking device's max frequency and loading it into device

max_frequency_str = subprocess.check_output ("adb shell su -c 'cat /sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq'", shell=True).decode('utf-8')
max_frequency = int(max_frequency_str.strip())

In [161]:
max_frequency

1593600

In [162]:
cmd = "adb shell \"su -c 'echo %d > /sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'\"" % max_frequency
subprocess.check_output (cmd , shell=True).decode('utf-8')

''

In [163]:
cmd = "adb shell \"su -c 'cat /sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'\""
subprocess.check_output (cmd , shell=True).decode('utf-8').strip()

''

In [164]:
#checking display if it is turned on. If it isn't then turn on the display

cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

if 'false' in ret:
    print("Display is turned off, try to turn on")
    cmd = "adb shell input keyevent 26"
    ret = subprocess.check_output (cmd , shell=True).decode('utf-8')
else:
    print("Display is already turned on")

Display is already turned on


In [165]:
#checking the display whether it is turned on/off

cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

print(ret)

true


cmd = "adb shell su -c 'cat /sys/devices/13900000.dsim/backlight/panel/device/lcd/panel/adaptive_control'"
auto_brightness_str = subprocess.check_output (cmd, shell=True).decode('utf-8')
is_auto_brightness_on = int(auto_brightness_str.strip())

if is_auto_brightness_on == 1:
    print("Auto brightness feature is turned on, try to turn off")
    cmd = "adb shell \"su -c 'echo 0 > /sys/devices/13900000.dsim/backlight/panel/device/lcd/panel/adaptive_control'\""
    subprocess.check_output (cmd , shell=True).decode('utf-8')
else:
    print("Auto brightness feature is already turned off")

In [166]:
#checking brightness of device and setting it into the brightest brightness possible

cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/brightness'"
brightness_str = subprocess.check_output (cmd, shell=True).decode('utf-8')
brightness = int(brightness_str.strip())

In [167]:
brightness

255

In [168]:
cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/max_brightness'"
max_brightness_str = subprocess.check_output (cmd, shell=True).decode('utf-8')
max_brightness = int(max_brightness_str.strip())

In [169]:
if brightness != max_brightness:
    print("Brightness level doesn't equal to maximum level")
    cmd = "adb shell \"su -c 'echo %d > /sys/devices/13900000.dsim/backlight/panel/brightness'\"" % max_brightness
    subprocess.check_output (cmd , shell=True).decode('utf-8')
else:
    print("Brightness level is already equal to maximum level")

Brightness level is already equal to maximum level


In [170]:
cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/brightness'"
subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

'255'

In [171]:
#for easy use - function to set brightness to maximum level

def set_brightness_to_maximum():
    cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/brightness'"
    brightness_str = subprocess.check_output (cmd, shell=True).decode('utf-8')
    brightness = int(brightness_str.strip())
    
    cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/max_brightness'"
    max_brightness_str = subprocess.check_output (cmd, shell=True).decode('utf-8')
    max_brightness = int(max_brightness_str.strip())
    
    if brightness != max_brightness:
        print("Brightness level doesn't equal to maximum level")
        cmd = "adb shell \"su -c 'echo %d > /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/brightness'\"" % max_brightness
        subprocess.check_output (cmd , shell=True).decode('utf-8')
    else:
        print("Brightness level is already equal to maximum level")
    
    cmd = "adb shell su -c 'cat /sys/devices/soc/900000.qcom,mdss_mdp/900000.qcom,mdss_mdp:qcom,mdss_fb_primary/leds/lcd-backlight/brightness'"
    print(subprocess.check_output(cmd, shell=True).decode('utf-8').strip())

<h3>Find image files to measure power consumptions

In [172]:
import re

#images path
DATASET_PATH="/sdcard/Amoled/dataset_train"

In [218]:
#find image files that names matches pattern
raw_output = subprocess.check_output ('adb shell ls %s' % DATASET_PATH, shell=True).decode('utf-8')

pattern= re.compile("r[0-9]{3}_g[0-9]{3}_b[0-9]{3}.jpg")
image_filenames = sorted(pattern.findall(raw_output))

In [219]:
image_filenames

['r000_g000_b000.jpg',
 'r000_g000_b089.jpg',
 'r000_g000_b124.jpg',
 'r000_g000_b149.jpg',
 'r000_g000_b170.jpg',
 'r000_g000_b188.jpg',
 'r000_g000_b203.jpg',
 'r000_g000_b218.jpg',
 'r000_g000_b231.jpg',
 'r000_g000_b243.jpg',
 'r000_g000_b255.jpg',
 'r000_g089_b000.jpg',
 'r000_g089_b089.jpg',
 'r000_g089_b124.jpg',
 'r000_g089_b149.jpg',
 'r000_g089_b170.jpg',
 'r000_g089_b188.jpg',
 'r000_g089_b203.jpg',
 'r000_g089_b218.jpg',
 'r000_g089_b231.jpg',
 'r000_g089_b243.jpg',
 'r000_g089_b255.jpg',
 'r000_g124_b000.jpg',
 'r000_g124_b089.jpg',
 'r000_g124_b124.jpg',
 'r000_g124_b149.jpg',
 'r000_g124_b170.jpg',
 'r000_g124_b188.jpg',
 'r000_g124_b203.jpg',
 'r000_g124_b218.jpg',
 'r000_g124_b231.jpg',
 'r000_g124_b243.jpg',
 'r000_g124_b255.jpg',
 'r000_g149_b000.jpg',
 'r000_g149_b089.jpg',
 'r000_g149_b124.jpg',
 'r000_g149_b149.jpg',
 'r000_g149_b170.jpg',
 'r000_g149_b188.jpg',
 'r000_g149_b203.jpg',
 'r000_g149_b218.jpg',
 'r000_g149_b231.jpg',
 'r000_g149_b243.jpg',
 'r000_g149

<h3>Function for showing images to device's display

In [174]:
#viewer.apk needs to be installed
VIEWER_APP = "com.yhchoi.mobed.yhviewer"

In [203]:
def show_image(filename):
    wait_adb_connection()
    
    # Stop Mtp Service which can make fluctuations in measurement results
    #cmd = "adb shell su -c 'pm disable com.samsung.android.MtpApplication'"
    #subprocess.check_output (cmd, shell=True).decode('utf-8')
    #time.sleep(0.1)
    
    while True:
        try:
            cmd = "adb shell am force-stop %s" % VIEWER_APP
            subprocess.check_output (cmd, shell=True).decode('utf-8')
            time.sleep(0.5)
            
            cmd = 'adb shell am start -n "com.yhchoi.mobed.yhviewer/com.yhchoi.mobed.yhviewer.MainActivity" -a android.intent.action.MAIN -c android.intent.category.LAUNCHER -d "{}"'.format(filename)
            subprocess.check_output (cmd, shell=True).decode('utf-8')
            time.sleep(0.5)
            
            break
        except:
            wait_adb_connection()
            continue

<h3>Measure the base power of the display

In [176]:
def turn_on_display():
    cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
    ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

    if 'false' in ret:
        print("Display is turned off, try to turn on")
        cmd = "adb shell input keyevent 26"
        ret = subprocess.check_output (cmd , shell=True).decode('utf-8')
    else:
        print("Display is already turned on")
    
    cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
    ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

In [177]:
def turn_off_display():
    cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
    ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

    if 'true' in ret:
        print("Display is turned on, try to turn off")
        cmd = "adb shell input keyevent 26"
        ret = subprocess.check_output (cmd , shell=True).decode('utf-8')
    else:
        print("Display is already turned off")
        
    cmd = "adb shell dumpsys deviceidle | grep mScreenOn"
    ret = subprocess.check_output (cmd , shell=True).decode('utf-8').strip().split('=')[1]

In [178]:
def measure_base_power():
    turn_off_display()
    power_before = measure_power()
    
    wait_adb_connection()
    turn_on_display()
    
    set_brightness_to_maximum()
    image_path = "{}/{}".format(DATASET_PATH, image_filenames[0])
    show_image(image_path)
    
    power_after = measure_power()
    
    return (min(power_after) - min(power_before))

In [182]:
base_power = measure_base_power()

Display is turned on, try to turn off


wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
data sequence skipped, lost packet?


Display is turned off, try to turn on
Brightness level is already equal to maximum level
255


wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
data sequence skipped, lost packet?


In [183]:
base_power

191.3070599836957

<h3>Start measuring

!pip install progressbar2

In [186]:
import progressbar

In [187]:
def get_RGB(filename):
    name = filename.split('.')[0]
    
    rgb_raw = name.split('_')
    
    r = int(rgb_raw[0][1:])
    g = int(rgb_raw[1][1:])
    b = int(rgb_raw[2][1:])
    
    return r,g,b

In [204]:
measured_powers_mW = list()

bar = progressbar.ProgressBar()
for i in bar(range(len(image_filenames))):
    fname = image_filenames[i]
    
    wait_adb_connection()
    
    image_path = "{}/{}".format(DATASET_PATH, fname)
    show_image(image_path)
    
    power = measure_power()
    
    sample = []
    sample.extend(get_RGB(fname))
    sample.append(min(power))  
    measured_powers_mW.append(sample)

N/A% (0 of 1331) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
data sequence skipped, lost packet?
  0% (1 of 1331) |                       | Elapsed Time: 0:00:13 ETA:   5:01:00wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
data sequence skipped, lost packet?
  0% (2 of 1331) |                       | Elapsed Time: 0:00:29 ETA:   5:47:26wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
data sequence skipped, lost packet?
  0% (3 of 1331) |                       | Elapsed Time: 0:00:45 ETA:   5:54:13wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10, len=58
wanted data, dropped type=0x10

In [213]:
measured_powers_mW[2]

[188, 188, 203, 1051.3497013934968]

In [220]:
# !pip install pandas
from pandas import DataFrame

colume_names = ['red', 'green', 'blue', 'power']
power_df = DataFrame(measured_powers_mW, columns = colume_names)
power_df.loc[lambda x: x.red == 255]
power_df.to_csv("marlin_default_br255_n1331.csv")